# Задача регрессии

In [88]:
import pandas as pd
import numpy as np

from sklearn.metrics import r2_score
from sklearn import linear_model
from geopy.distance import vincenty
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures

# Подготовка данных
## Интеграция данных

In [57]:
df_vacancy = pd.read_csv('hh_vacancy.csv')
df_vacancy = df_vacancy.drop_duplicates('id')
df_vacancy

,id,area_id,area_name,employer_id,employment_id,employment_name,experience_id,experience_name,key_skills,name,published_at,salary_currency,salary_from,salary_to,schedule_id,schedule_name,specialization_id,specialization_name
0,25607531,1,Москва,1747017.0,full,Полная занятость,between3And6,От 3 до 6 лет,"PHP, Symfony, PostgreSQL, Redis, Design Patter...",Senior/Middle PHP-Developer,2018-05-06T17:43:45+0300,RUR,120000.0,170000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
1,25435136,1,Москва,553714.0,project,Проектная работа,between3And6,От 3 до 6 лет,NaN,Ведущий фронтенд-разработчик,2018-05-05T14:43:01+0300,NaN,NaN,NaN,remote,Удаленная работа,1,"Информационные технологии, интернет, телеком"
2,25284500,84,Ставрополь,5402.0,full,Полная занятость,between3And6,От 3 до 6 лет,"JavaScript, HTML5, CSS3, AngularJS, Bootstrap",Frontend-разработчик,2018-05-04T09:24:18+0300,RUR,60000.0,80000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
3,25641559,1,Москва,1554.0,full,Полная занятость,between1And3,От 1 года до 3 лет,"1С программирование, 1С: Бухгалтерия, 1С Управ...",Программист-консультант 1С,2018-05-06T10:49:05+0300,NaN,NaN,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
4,25527035,2,Санкт-Петербург,3289100.0,full,Полная занятость,between1And3,От 1 года до 3 лет,NaN,Менеджер по продажам,2018-05-05T16:15:16+0300,RUR,35000.0,100000.0,flexible,Гибкий график,1,"Информационные технологии, интернет, телеком"
5,25178591,1,Москва,2353123.0,full,Полная занятость,between3And6,От 3 до 6 лет,NaN,Системный аналитик,2018-05-07T00:54:39+0300,NaN,NaN,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
6,25449125,1,Москва,934019.0,full,Полная занятость,between3And6,От 3 до 6 лет,NaN,Java-разработчик,2018-05-06T11:41:28+0300,RUR,120000.0,220000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
7,25459894,1,Москва,1053428.0,full,Полная занятость,between3And6,От 3 до 6 лет,"Linux, AWS, Proxmox, Vagrant, Docker, Админист...",Системный администратор (DevOps),2018-05-06T17:00:04+0300,RUR,100000.0,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
8,25041175,2,Санкт-Петербург,934019.0,full,Полная занятость,between1And3,От 1 года до 3 лет,NaN,Java разработчик (middle/senior),2018-05-05T18:43:16+0300,RUR,120000.0,220000.0,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"
9,25295125,78,Самара,2618807.0,full,Полная занятость,between1And3,От 1 года до 3 лет,"Коммуникабельность, Аналитический склад ума, Н...",Менеджер по работе с корпоративными клиентами,2018-05-06T11:45:27+0300,RUR,35000.0,NaN,fullDay,Полный день,1,"Информационные технологии, интернет, телеком"


In [58]:
df_vacancy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19837 entries, 0 to 19836
Data columns (total 18 columns):
id                     19837 non-null int64
area_id                19837 non-null int64
area_name              19837 non-null object
employer_id            19813 non-null float64
employment_id          19805 non-null object
employment_name        19805 non-null object
experience_id          19805 non-null object
experience_name        19805 non-null object
key_skills             6654 non-null object
name                   19837 non-null object
published_at           19837 non-null object
salary_currency        10358 non-null object
salary_from            9611 non-null float64
salary_to              4594 non-null float64
schedule_id            19805 non-null object
schedule_name          19805 non-null object
specialization_id      19837 non-null int64
specialization_name    19837 non-null object
dtypes: float64(3), int64(3), object(12)
memory usage: 2.9+ MB


In [59]:
# Разделение на обучающую и тестовую выборку по id, который соответствует порядку добавления вакаансий на сайт.
count_df = len(df_vacancy)
df_vacancy.sort_values(by='id', ascending=True, inplace=True)
df_vacancy_test = df_vacancy[int(2*count_df/3):]
df_vacancy_train = df_vacancy[:int(2*count_df/3)]

In [60]:
# Удаление записей с пропущенными значениями NaN.
def df_clean_nan_id(df):
    return df[np.isfinite(df['salary_from']) & 
              np.isfinite(df['area_id']) & 
              np.isfinite(df['employer_id']) &
              np.isfinite(df['specialization_id'])]

df_vacancy_train = df_clean_nan_id(df_vacancy_train)

In [61]:
# Таблица с компаниями.
df_company = pd.read_csv('hh_company.csv')
df_company = df_company.drop_duplicates('employer_id')
df_company.head(2)

,employer_id,employer_name,employer_site,industry_id,industry_name
0,1747017.0,Колтач Солюшнс,NaN,7,"Информационные технологии, системная интеграци..."
1,553714.0,Нотамедиа,NaN,7,"Информационные технологии, системная интеграци..."


In [62]:
# Объдединение вакансий с компаниями.
def df_merge_vacancy(df):
    return pd.merge(df, df_company, on='employer_id')

df_vacancy_train = df_merge_vacancy(df_vacancy_train)
df_vacancy_train.head(2)

,id,area_id,area_name,employer_id,employment_id,employment_name,experience_id,experience_name,key_skills,name,...,salary_from,salary_to,schedule_id,schedule_name,specialization_id,specialization_name,employer_name,employer_site,industry_id,industry_name
0,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,35000.0,NaN,flexible,Гибкий график,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци..."
1,7296077,67,Великий Новгород,558625.0,full,Полная занятость,noExperience,Нет опыта,NaN,Специалист отдела продаж,...,30000.0,60000.0,fullDay,Полный день,17,Продажи,Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци..."


In [63]:
# Таблица с городами.
df_cities = pd.read_csv('cities.csv')
df_cities.head(2)

,Индекс,Тип региона,Регион,Тип района,Район,Тип города,Город,Тип н/п,Н/п,Код КЛАДР,...,Уровень по ФИАС,Признак центра района или региона,Код ОКАТО,Код ОКТМО,Код ИФНС,Часовой пояс,Широта,Долгота,Федеральный округ,Население
0,385200.0,Респ,Адыгея,NaN,NaN,г,Адыгейск,NaN,NaN,100000200000,...,4: город,0,79403000000,7.970300e+10,107,UTC+3,44.878372,39.190172,Южный,12689.0
1,385000.0,Респ,Адыгея,NaN,NaN,г,Майкоп,NaN,NaN,100000100000,...,4: город,2,79401000000,7.970100e+10,105,UTC+3,44.609827,40.100653,Южный,144055.0


In [64]:
# Формированеи признаков для городов.
df_cities = df_cities[['Город','Широта','Долгота','Население', 'Регион']]
df_cities['Город'] = df_cities.apply(lambda row: row['Регион']
                                     if type(row['Город']) != str and np.isnan(row['Город'])
                                     else row['Город'], axis=1)
df_cities = df_cities[['Город','Широта','Долгота','Население']]
df_cities = df_cities.rename(index=str, columns={"Город": "area_name"})
df_cities = df_cities.rename(index=str, columns={"Широта": "latitude"})
df_cities = df_cities.rename(index=str, columns={"Долгота": "longitude"})
df_cities = df_cities.rename(index=str, columns={"Население": "population"})
df_cities['population'].fillna((df_cities['population'].mean()), inplace=True)
moscow_latitude, moscow_longitude = list(df_cities.loc[df_cities.area_name == 'Москва'][['latitude','longitude']].iloc[0])
df_cities['distance_to_Moscow'] = df_cities.apply(lambda row: vincenty((moscow_latitude, moscow_longitude),
                                                                       (row.latitude, row.longitude)).kilometers, axis=1)
df_cities.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning:

Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.



,area_name,latitude,longitude,population,distance_to_Moscow
0,Адыгейск,44.878372,39.190172,12689.0,1214.804843
1,Майкоп,44.609827,40.100653,144055.0,1251.957644


In [65]:
# Объдединение вакансий с городами.
def df_merge_city(df):
    return pd.merge(df, df_cities, on='area_name')

df_vacancy_train = df_merge_city(df_vacancy_train)
df_vacancy_train.head(2)

,id,area_id,area_name,employer_id,employment_id,employment_name,experience_id,experience_name,key_skills,name,...,specialization_id,specialization_name,employer_name,employer_site,industry_id,industry_name,latitude,longitude,population,distance_to_Moscow
0,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,3,"Маркетинг, реклама, PR",Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци...",59.939125,30.315822,4848742.0,636.038853
1,10161653,2,Санкт-Петербург,558625.0,full,Полная занятость,noExperience,Нет опыта,NaN,Менеджер call-центра,...,17,Продажи,Cистема Северо-Запад,NaN,7,"Информационные технологии, системная интеграци...",59.939125,30.315822,4848742.0,636.038853


In [66]:
#Приводим к одной валюте
def convert_valute_to_rub(salary, currency):
    if currency == 'USD':
        return salary*65.58
    elif currency == 'EUR':
        return salary*73.66
    elif currency == 'UAH':
        return salary*2.41
    return salary

def df_convert_valute(df):
    df['salary_from'] = df.apply(lambda row: convert_valute_to_rub(row.salary_from, row.salary_currency), axis=1)
    return df

print(df_vacancy_train['salary_currency'].unique())
df_vacancy_train = df_convert_valute(df_vacancy_train)

['RUR' 'EUR' 'USD' 'UAH']


## Формирование признаков

In [67]:
# Добавление информации о кол-ве ключевых навыков вакансии.
def df_add_key_skills_count(df):
    df['key_skills_count'] = df.apply(lambda row:0 if type(row.key_skills) == float
                                      else row.key_skills.count(',') + 1, axis=1)
    return df

df_vacancy_train = df_add_key_skills_count(df_vacancy_train)
df_vacancy_train['key_skills_count'].describe()

count    6186.000000
mean        2.503556
std         4.532688
min         0.000000
25%         0.000000
50%         0.000000
75%         4.000000
max        30.000000
Name: key_skills_count, dtype: float64

### One Hot Encoding

In [68]:
def one_hot_encode(df, df_train):
    categorical_columns = [
        'employment_name',
        'experience_name',
        'salary_currency',
        'schedule_name'
    ]
    for category in categorical_columns:
        for category_value in df_train[category].unique():
            df[f'{category}_is_{category_value}'] = (df[category] == category_value).astype(float)
    return df

df_vacancy_train = one_hot_encode(df_vacancy_train, df_vacancy_train)
df_vacancy_train.head(2)

,id,area_id,area_name,employer_id,employment_id,employment_name,experience_id,experience_name,key_skills,name,...,experience_name_is_Более 6 лет,salary_currency_is_RUR,salary_currency_is_EUR,salary_currency_is_USD,salary_currency_is_UAH,schedule_name_is_Гибкий график,schedule_name_is_Полный день,schedule_name_is_Сменный график,schedule_name_is_Удаленная работа,schedule_name_is_Вахтовый метод
0,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,10161653,2,Санкт-Петербург,558625.0,full,Полная занятость,noExperience,Нет опыта,NaN,Менеджер call-центра,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Target Encoding

In [69]:
areas = df_vacancy_train[['id', 'area_name']].\
        groupby(['area_name']).\
        agg({'id': 'count'}).\
        rename(columns={'id': 'area_vacancy_count'})
areas.head()

,area_vacancy_count
area_name,
Абакан,6
Агрыз,2
Александров,2
Алексин,2
Альметьевск,10


In [70]:
areas = df_vacancy_train[['salary_from', 'area_name']].\
        groupby(['area_name']).\
        agg({'salary_from': 'mean'}).\
        rename(columns={'salary_from': 'area_salary_mean'})
areas.head()

,area_salary_mean
area_name,
Абакан,38166.666667
Агрыз,13500.000000
Александров,35000.000000
Алексин,39000.000000
Альметьевск,35200.000000


In [71]:
def target_encode_mean(df, df_train):
    categorical_columns = [
        'area_name',
        'employer_id',
        'employment_name',
        'experience_name',
        'salary_currency',
        'schedule_name'
    ]
    for category in categorical_columns:
        category_table = df_vacancy_train[['salary_from', category]].\
                            groupby([category]).\
                            agg({'salary_from': 'mean'}).\
                            rename(columns={'salary_from': f'{category}_salary_mean'})
        df = pd.merge(df, category_table, left_on=category, right_index=True)
    return df

def target_encode_count(df, df_train):
    categorical_columns = [
        'area_name',
        'employer_id',
        'employment_name',
        'experience_name',
        'salary_currency',
        'schedule_name'
    ]
    for category in categorical_columns:
        category_table = df_vacancy_train[['id', category]].\
                            groupby([category]).\
                            agg({'id': 'count'}).\
                            rename(columns={'id': f'{category}_id_count'})
        df = pd.merge(df, category_table, left_on=category, right_index=True)
    return df


df_vacancy_train = target_encode_mean(df_vacancy_train, df_vacancy_train)
df_vacancy_train = target_encode_count(df_vacancy_train, df_vacancy_train)
df_vacancy_train.head(2)

,id,area_id,area_name,employer_id,employment_id,employment_name,experience_id,experience_name,key_skills,name,...,employment_name_salary_mean,experience_name_salary_mean,salary_currency_salary_mean,schedule_name_salary_mean,area_name_id_count,employer_id_id_count,employment_name_id_count,experience_name_id_count,salary_currency_id_count,schedule_name_id_count
0,7160726,2,Санкт-Петербург,558625.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Оператор call-центра,...,24043.493151,39471.266614,52531.193002,63035.525926,543,13,146,2543,6145,405
106,19356190,2,Санкт-Петербург,43410.0,part,Частичная занятость,noExperience,Нет опыта,NaN,Менеджер по продажам,...,24043.493151,39471.266614,52531.193002,63035.525926,543,57,146,2543,6145,405


### Дополнительные параметры

Уровень жизни
https://www.domofond.ru/statya/polnyy_reyting_gorodov_rossii_po_kachestvu_zhizni_v_2018_godu/7679
    
Рейтинг компаний
http://www.cnews.ru/reviews/rynok_it_itogi_2017/review_table/5ea292822e4f02a7c71a3495f1e456e5b320f86a

In [72]:
# code

In [73]:
# code

In [74]:
# code

### Выбор признаков

In [75]:
df_vacancy_train.columns

Index(['id', 'area_id', 'area_name', 'employer_id', 'employment_id',
       'employment_name', 'experience_id', 'experience_name', 'key_skills',
       'name', 'published_at', 'salary_currency', 'salary_from', 'salary_to',
       'schedule_id', 'schedule_name', 'specialization_id',
       'specialization_name', 'employer_name', 'employer_site', 'industry_id',
       'industry_name', 'latitude', 'longitude', 'population',
       'distance_to_Moscow', 'key_skills_count',
       'employment_name_is_Частичная занятость',
       'employment_name_is_Полная занятость', 'employment_name_is_Стажировка',
       'employment_name_is_Проектная работа', 'experience_name_is_Нет опыта',
       'experience_name_is_От 1 года до 3 лет',
       'experience_name_is_От 3 до 6 лет', 'experience_name_is_Более 6 лет',
       'salary_currency_is_RUR', 'salary_currency_is_EUR',
       'salary_currency_is_USD', 'salary_currency_is_UAH',
       'schedule_name_is_Гибкий график', 'schedule_name_is_Полный день',
    

In [76]:
train_y = df_vacancy_train['salary_from']
x_labels = ['latitude', 'longitude', 'population',
       'distance_to_Moscow', 'key_skills_count',
       'employment_name_is_Частичная занятость',
       'employment_name_is_Полная занятость', 'employment_name_is_Стажировка',
       'employment_name_is_Проектная работа', 'experience_name_is_Нет опыта',
       'experience_name_is_От 1 года до 3 лет',
       'experience_name_is_От 3 до 6 лет', 'experience_name_is_Более 6 лет',
       'salary_currency_is_RUR', 'salary_currency_is_EUR',
       'salary_currency_is_USD', 'salary_currency_is_UAH',
       'schedule_name_is_Гибкий график', 'schedule_name_is_Полный день',
       'schedule_name_is_Сменный график', 'schedule_name_is_Удаленная работа',
       'schedule_name_is_Вахтовый метод', 'area_name_salary_mean',
       'employer_id_salary_mean', 'employment_name_salary_mean',
       'experience_name_salary_mean', 'salary_currency_salary_mean',
       'schedule_name_salary_mean', 'area_name_id_count',
       'employer_id_id_count', 'employment_name_id_count',
       'experience_name_id_count', 'salary_currency_id_count',
       'schedule_name_id_count']
train_X = df_vacancy_train[x_labels]

### Нормализация

MinMaxScaler, StandardScaler

In [77]:
min_max_scaler = MinMaxScaler()
print(min_max_scaler.fit(train_X))
train_X = min_max_scaler.transform(train_X)

MinMaxScaler(copy=True, feature_range=(0, 1))


## Обучение линейной регрессии

In [78]:
reg = linear_model.LinearRegression()
reg.fit(train_X, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [79]:
# Коэф. уравнения.
pd.DataFrame(list(zip(x_labels, reg.coef_)), columns = ['name', 'coef']).sort_values('coef', ascending=False)

,name,coef
6,employment_name_is_Полная занятость,1.363259e+17
17,schedule_name_is_Гибкий график,8.761313e+16
21,schedule_name_is_Вахтовый метод,8.717915e+16
8,employment_name_is_Проектная работа,6.100260e+16
5,employment_name_is_Частичная занятость,5.686568e+16
7,employment_name_is_Стажировка,5.358644e+16
19,schedule_name_is_Сменный график,5.227243e+16
33,schedule_name_id_count,5.135090e+16
25,experience_name_salary_mean,4.356274e+16
20,schedule_name_is_Удаленная работа,4.354631e+16


### Оценка качества обучения

In [83]:
# Точность на обучающей выборке.
train_r2 = r2_score(train_y, reg.predict(train_X))
train_r2

0.7472275607200445

In [84]:
# Подготовка тестовой выборки.
df_vacancy_test = df_clean_nan_id(df_vacancy_test)
df_vacancy_test = df_merge_vacancy(df_vacancy_test)
df_vacancy_test = df_merge_city(df_vacancy_test)
df_vacancy_test = df_convert_valute(df_vacancy_test)
df_vacancy_test = df_add_key_skills_count(df_vacancy_test)
df_vacancy_test = one_hot_encode(df_vacancy_test, df_vacancy_train)
df_vacancy_test = target_encode_mean(df_vacancy_test, df_vacancy_train)
df_vacancy_test = target_encode_count(df_vacancy_test, df_vacancy_train)

In [85]:
test_y = df_vacancy_test['salary_from']
test_X = df_vacancy_test[x_labels]
test_X = min_max_scaler.transform(test_X)
# Точность на тестовой выборке.
test_r2 = r2_score(test_y, reg.predict(test_X))
test_r2

0.502046050847845

## Что еще?

Полиномилальная регрессия

    from sklearn.preprocessing import PolynomialFeatures

Алгоритм *k*-ближайших соседей

    from sklearn.neighbors import KNeighborsRegressor

In [89]:
# Точность на обучающей выборке.
quadratic = PolynomialFeatures(degree=2)
train_X_quad = quadratic.fit_transform(train_X)

reg = linear_model.LinearRegression()
reg = reg.fit(train_X_quad, train_y)
train_r2 = r2_score(train_y, reg.predict(train_X_quad))
train_r2

0.7867004017550059

In [91]:
pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items',None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [92]:
# Коэф. уравнения.
pd.DataFrame(list(zip(quadratic.get_feature_names(x_labels), reg.coef_)),
             columns = ['name', 'coef']).sort_values('coef', ascending=False)

,name,coef
313,experience_name_is_Нет опыта schedule_name_is_Гибкий график,2.968031e+17
329,experience_name_is_Нет опыта schedule_name_id_count,1.582952e+17
21,schedule_name_is_Удаленная работа,1.044328e+17
337,experience_name_is_От 1 года до 3 лет schedule_name_is_Гибкий график,8.513934e+16
626,experience_name_id_count schedule_name_id_count,8.201382e+16
245,employment_name_is_Полная занятость schedule_name_salary_mean,7.229652e+16
370,experience_name_is_От 3 до 6 лет schedule_name_salary_mean,7.089092e+16
507,schedule_name_is_Полный день experience_name_id_count,6.872389e+16
347,experience_name_is_От 1 года до 3 лет schedule_name_salary_mean,6.233356e+16
247,employment_name_is_Полная занятость employer_id_id_count,5.812028e+16
